# Continuous-time Markov processes

A **Markov process** is a stochastic model for a system, in which the evolution of the system is determined only by the _current_ state of the system, not by any state in the past _(except perhaps through derivatives of the dynamical quantities, where those derivatives exist - often they do not)_. Such a system may be said to possess the **Markov property** ([Wikipedia](https://en.wikipedia.org/wiki/Markov_property)), or to be 'memoryless'.

The most famous and elementary types of Markov processes are Markov chains ([Wikipedia](https://en.wikipedia.org/wiki/Markov_chain)), which have a discrete set of states (which may be infinite) and evolution between the states takes place in discrete, uniform units of time. However, for some systems a **continuous-time Markov process** may be more appropriate. Under continuous-time evolution, we speak not of 'transition probabilities' but rather of 'transition rates'. This is most important when we care about the dependence of the dynamical properties (_e.g._ the population of a species) on time, and when the rate of change depends on the quantity (_e.g._ higher population implies shorter periods between birth or death events).

In the examples below, the dynamical quantities are still assumed to take discrete values; it is only the time between state evolutions that is assumed to be continuous. This makes these CTMPs different from Brownian processes, where the dynamical quantities (_e.g._ position of a particle) are also continuous. 

At each state, the transition rates to other possible states are given by specifying a transition function, and typically these transitions are assumed to be Poisson processes - with the first event to 'trigger' determining the next state that is visited. The time to transition is therefore distributed according to an exponential distribution.


In [ ]:
import logging
import random
import warnings

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
from scipy.interpolate import interp1d

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

logging.getLogger().setLevel(logging.WARN)

One way to numerically simulate a CTMP is to use an _event-driven simulation_. At a given state, the transition rates are calculated (as specified by the transition function); and random exponential variables are sampled, one for each of the transition rates. These r.v.s are the times until each of the possible transitions triggers. The 'shortest time wins' and the state evolves to the corresponding state. 

To avoid numerical overflow or underflow, we terminate the solution if transition rates become too large (and therefore evolution times too small).

In [ ]:
def evolution(fun, tspan, state0, *, args=(), t_eval=None, maxrate=1e6):
    """Event-driven simulation of a continuous-time Markov process."""
    # TODO specify max rate/min step, to interrupt in case of very high rates
    tmin, tmax = tspan
    
    state = state0
    t = tmin
    ts = [t]
    history = [state0]
    
    while t < tmax:
        transitions = fun(t, state, *args)
        transitions = {k: transitions[k] for k in transitions if transitions[k] > 0}
        if not transitions:
            break
            
        if maxrate:
            transitions = {k: transitions[k] for k in transitions if transitions[k] < maxrate}
            if not transitions:
                warnings.warn(
                    f"Maximum rate {maxrate} exceeded by all possible transitions at time {t}.", 
                    RuntimeWarning
                )
                break
                
        logging.info(transitions)

        
        times_to_move = {k: np.random.exponential(1/transitions[k]) for k in transitions}
        logging.info(times_to_move)       

        new_state = min(times_to_move, key=lambda k: times_to_move[k])
        time_elapsed = times_to_move[new_state]
        t += time_elapsed
        state = new_state
        ts.append(t)
        history.append(state)
            
    ts = np.array(ts)
    history = np.array(history)
    
    if t_eval is None:
        return ts, history
    
    interp = interp1d(ts, history, kind='previous', axis=0)
    return np.array(t_eval), np.array([interp(t) for t in t_eval])

## Stochastic Lotka-Volterra

The [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) are a family of simple models for the populations of a predator-prey system. The original model is a deterministic dynamical system, consisting of a pair of ordinary differential equations (continuous time, continuous populations). This model is suitable for large populations but leads to the 'atto-fox' problem: populations never become extinct but reach extremely small quantities that are somehow able to bounce back. One resolution is to turn the populations into discrete quantities (you can't have half a fox, or $10^{-18}$ foxes – but you can rescale units to talk in terms of 1000s of foxes, for example). Having done that we can also add uncertainty by introducing a transition function between states.

In [ ]:
def lv_transition_fun(t, n):
    x, y = n
    s = .001
    return {
        (x + s, y): 2/3 * x,
#         (x - s, y): x * x / 20,
        (x - 4/3*s, y+s): x * y,
        (x, y - s): y
    }

In [ ]:
@interact()
def lv_demo():
    ts, history = evolution(lv_transition_fun, (0, 25000), [1, 0.75],
                        t_eval=np.linspace(0, 25000, 101),
                        maxrate=10)
    prey, predators = history[:, 0], history[:, 1]
    fig = plt.figure(figsize=(8, 6))
    plt.plot(ts, prey, 'g-',
             ts, predators, 'r-')
    plt.grid()
    ax = plt.gca()
    ax.set_ylim([0, None])
    ax.legend(['prey', 'predators'])
    plt.show()

    fig = plt.figure(figsize=(15, 4))
    plt.plot(prey, predators, 'k')
    plt.grid()
    ax = plt.gca()
    ax.set_aspect('equal')
    ax.set_xlabel('prey')
    ax.set_ylabel('predators')
    ax.set_xlim([0, None])
    ax.set_ylim([0, None])
    plt.show()

While the basic, deterministic Lotka-Volterra system has solutions that are closed loops, the realisations of the stochastic system also exhibit periodic-like behaviour, but with some variation and therefore not closed loops.

If we calculate the _ensemble average_ of several realisations (perhaps with different initial conditions), we will find that the average also executes very periodic behaviour.

In [ ]:
@interact()
def lv_ensemble_demo():
    # Same initial conditions
    n_states = 5
    initial_states = [(1, 1) for _ in range(n_states)]


    # # Or different initial conditions
    # initial_states = [(1 + r*np.cos(th), 1 + r*np.sin(th)) 
    #                   for r in np.linspace(.1, .4, 4)
    #                   for th in np.linspace(0, 2*np.pi, 9)[:-1]]

    tmax = 25000
    ts = np.linspace(0, tmax, 201)

    histories = [evolution(lv_transition_fun, (0, tmax), s, t_eval=ts,
                          maxrate=20)[1] for s in initial_states]

    average_x = np.average([h[:, 0] for h in histories], axis=0)
    average_y = np.average([h[:, 1] for h in histories], axis=0)

    @interact(step=widgets.IntSlider(min=0, max=len(ts)-1, value=0, continuous_update=False))
    def plot_evolution(step):
        final_states = [h[1, :] for h in histories]
        plt.figure(figsize=[8,8])
        plt.plot(
            average_x, average_y, 'r-',
            [x[step][0] for x in histories], 
            [x[step][1] for x in histories],
            'k.',
            average_x[step], average_y[step], 'ro'
        )
        plt.gca().set_xlim([-.1, 2])
        plt.gca().set_ylim([-.1, 2])
        plt.gca().set_aspect('equal')
        plt.gca().grid()

## Stochastic harmonic oscillator

In [ ]:
def sho_transition_fun(t, s):
    x, y = s
    return {
        (x + 1, y): y,
        (x - 1, y): -y,
        (x, y + 1): -x,
        (x, y - 1): x
    }

In [ ]:
@interact()
def sho_demo():
    tmax = 100
    ts, history = evolution(sho_transition_fun, (0, tmax), [40, 40],
                        t_eval=np.linspace(0, tmax, 121))
    plt.plot(ts, history)
    plt.show()

    plt.plot(history[:, 0], history[:, 1])
    plt.show()

Although each individual realisation is noisy, the ensemble average again shows remarkable periodicity. However, unlike the Lotka-Volterra system, here there is quite a lot of variance.

In [ ]:
@interact()
def sho_ensemble_demo():
    initial_states = [(30 + r*np.cos(th), 40 + r*np.sin(th)) 
                  for r in np.linspace(1, 5, 5)
                  for th in np.linspace(0, 2*np.pi, 9)]

    histories = [evolution(sho_transition_fun, (0, 25), s, t_eval=np.linspace(0, 25, 161))[1]
                 for s in initial_states]
    average_x = np.average([h[:, 0] for h in histories], axis=0)
    average_y = np.average([h[:, 1] for h in histories], axis=0)

    @interact(step=widgets.IntSlider(min=0, max=len(histories[0])-1, value=0, continuous_update=False))
    def plot_evolution(step):
        final_states = [h[step][1] for h in histories]

        plt.figure(figsize=[8,8])
        plt.plot(
            average_x, average_y, 'r-',
            [x[step, 0] for x in histories],
            [x[step, 1] for x in histories],
                'k.',
            average_x[step], average_y[step], 'ro',)
        plt.gca().set_xlim([-60, 60])
        plt.gca().set_ylim([-60, 60])
        plt.gca().set_aspect('equal')
        plt.gca().grid()
    

## Decay with regeneration

In [ ]:
def transition_fun(t, s):
    x, y = s
    return {
        (x + 1, y): 1 - 0.05*x + y,
        (x - 1, y): 1 + 0.05*x - y,
        (x, y + 1): 1 - 0.05*y - x,
        (x, y - 1): 1 + 0.05*y + x
    }

ts, history = evolution(transition_fun, (0, 100), [10, 10],
                    t_eval=np.linspace(0, 100, 151))

plt.figure(figsize=(14, 5))
plt.plot(ts, history)
plt.show()

plt.plot(history[:, 0], history[:, 1])
plt.show()

In [ ]:
# Warning: This is slow.

initial_states = [(60 + r*np.cos(th), 60 + r*np.sin(th)) 
                  for r in np.linspace(1, 5, 5)
                  for th in np.linspace(0, 2*np.pi, 9)[:-1]]

histories = [evolution(transition_fun, (0, 100), s, t_eval=np.linspace(0, 100, 251))[1] for s in initial_states]
average_x = np.average([h[:, 0] for h in histories], axis=0)
average_y = np.average([h[:, 1] for h in histories], axis=0)

In [ ]:
@interact(step=widgets.IntSlider(min=0, max=len(histories[0])-1, value=0, continuous_update=False))
def plot_evolution(step):
    final_states = [h[step][1] for h in histories]

    plt.figure(figsize=[8,8])
    plt.plot(
        average_x, average_y, 'r-',
        [x[step, 0] for x in histories],
        [x[step, 1] for x in histories],
            'k.',
        average_x[step], average_y[step], 'ro',
    )
    plt.gca().set_xlim([-120, 120])
    plt.gca().set_ylim([-120, 120])
    plt.gca().set_aspect('equal')
    plt.gca().grid()
    

## A singular process

This is the stochastic version of the dynamical system
$$ \dot{s} = s^2, $$
which is a classic example of a system with a 'finite-time singularity', meaning that its solutions, which take the form
$$ s = \frac{1}{s_0^{-1} - t}, $$
reach infinity in a finite amount of time. Contrast this to $\dot{s} = s$, with solution $s = s_0 \exp t$, which diverges but only as $t \rightarrow \infty$.

The stochastic version also exhibits singular behaviour.

In [ ]:
@interact()
def singular_demo():
    def transition_fun(t, s):
        return {
            s + 0.01: 100 * s ** 2
        }

    s0 = 1
    ts, history = evolution(transition_fun, (0, 2), s0,
                       maxrate=1e8)

    plt.plot(ts, history,
            ts, 1/(1/s0 - ts), 'k--')
    ax = plt.gca()
    ax.set_yscale('log')
    ax.legend(['stochastic solution', 'continuous solution'])
    plt.show()

## Queueing process with multiple servers

Inspired when I was living at Minack with 40 other people with three showers between us.

In [ ]:
@interact(
    in_rate=widgets.FloatSlider(min=0, max=5, value=1, continuous_update=False),
    out_rate=widgets.FloatSlider(min=0, max=5, value=1, continuous_update=False),
    n_servers=widgets.IntSlider(min=0, max=12, value=3),
)
def queueing_process(in_rate, out_rate, n_servers):
    def transition_fun(t, s):
        todo, inprog, done = s
        outcomes = {}
        if inprog == n_servers:  # max capacity already, so new tickets go to the queue
            outcomes[todo + 1, inprog, done] = in_rate
        else:
            outcomes[todo, inprog + 1, done] = in_rate

        if todo > 0:
            outcomes[todo - 1, inprog, done + 1] = out_rate * inprog
        else:
            outcomes[0, inprog - 1, done + 1] = out_rate * inprog


        return outcomes

    ts, history = evolution(transition_fun, (0, 100), (0, 0, 0),
                        t_eval=np.linspace(0, 100, 101))

    ax = plt.gca()
    ax.plot(
        ts, history[:, 0], 'r-',
        ts, history[:, 1], 'k:',
#         ts, history[:, 2], 'k-'
    )
    ax.set_ylim([0, None])
    ax.grid()
    ax.legend(['queueing', 'in progress', 'done'])
#     return ts, history

## Immigration-death system (Mathematical Biology sheet 2 question 4)

Rates:
* $\lambda_1, \lambda_2$ - birth of one or two offspring
* $\beta n$ - deaths

In [ ]:
@interact(
    l_1=widgets.FloatSlider(min=0, max=5, value=1, continuous_update=False),
    l_2=widgets.FloatSlider(min=0, max=5, value=1, continuous_update=False),
    beta=widgets.FloatSlider(min=0, max=5, value=1, continuous_update=False),
)
def immigration_death_demo(l_1, l_2, beta):
    def transition_fun(t, s, l_1, l_2, beta):
        return {
            s + 1: l_1,
            s + 2: l_2,
            s - 1: beta * s
        }

    tmax = 500
    l_1, l_2, beta = 1, 1, 0.1
    ts, history = evolution(transition_fun, (0, tmax), 0,
                        args=(l_1, l_2, beta),
                        t_eval=np.arange(0, tmax, 1))
    
    mean = 1/beta * (l_1 + 2*l_2)
    sigma = (1/beta * (l_1 + 3*l_2))**0.5
    plt.plot(ts, history, 'r-',
             ts, [mean] * len(ts), 'k--',
             ts, [mean + sigma] * len(ts), 'k:',
             ts, [mean - sigma] * len(ts), 'k:',
    )
    plt.show()